In [1]:
import gym
import math
import random
import numpy as np
import pandas as pd

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

#env = gym.make('CartPole-v0').unwrapped

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# if gpu is to be used
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=2)

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward','content'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [3]:
class DQN(nn.Module):

    def __init__(self,inputs,outputs):
        super(DQN, self).__init__()
        self.conn1 = nn.Linear(inputs,16)
        self.bn1 = nn.BatchNorm1d(16)
        self.conn2 = nn.Linear(16,8)
        self.bn2 = nn.BatchNorm1d(8)
        self.conn3 = nn.Linear(8, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.float().to(device)
        x = F.relu(self.bn1(self.conn1(x)))
        x = F.relu(self.bn2(self.conn2(x)))
        return self.conn3(x)

    

In [4]:
data_path = '/home/ubuntu/data/dataset/R79619_U200_V5000/'
UIT = pd.read_csv(data_path + 'UIT.csv')
UIT

,u,i,day,video_type,level1,level2,level3,level4,time
0,3,3006,0,11,0,0,0,0,0
1,63,2232,0,11,0,0,0,0,0
2,68,2166,0,11,0,0,0,0,0
3,63,4578,0,11,0,0,0,0,0
4,78,1821,0,47,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
79614,133,3911,29,13,0,0,0,0,43197
79615,133,1684,29,13,0,0,0,0,43198
79616,185,1552,29,11,0,0,0,0,43198
79617,61,4078,29,13,0,0,0,0,43199


In [5]:
trainUIT = UIT[UIT['day']<18]
contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())

In [6]:
class ENV(object):
    def __init__(self,userNum,contentNum):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum)
        self.e = np.zeros(shape=contentNum)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l_edge = 0.1
        self.l_cp = 1

        self.B = np.full(shape=userNum,fill_value=10,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):

        self.p = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p

        self.r[u] = self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (torch.from_numpy(self.r),
                torch.from_numpy(self.p) , 
                torch.from_numpy(self.e),
                torch.from_numpy(self.S),
                self.l_edge,
                self.l_cp)

    def reset(self):
        self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
        self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
        self.e = np.zeros(shape=self.contentNum)
        self.S = np.ones(shape=self.contentNum,dtype=int)
        self.l_edge = 0.1
        self.l_cp = 1
        self.B = np.full(shape=self.userNum,fill_value=10,dtype=int)
        self.pipe = collections.OrderedDict()


In [7]:
#每个神经网络单独作为一个reward进行训练
class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = torch.zeros(size=(env.contentNum,),dtype=int)

        self.Bu = int(env.B[self.u])
        self.contentNum = env.contentNum
        self.userNum = env.userNum

        self.r , self.p , self.e, self.S,self.l_edge, self.l_cp = env.getStatus()

        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.lastAction = self.action

        self.reward = 0
        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']
        self.EPS_START = 0.9
        self.EPS_END = 0.1
        self.EPS_DECAY = 10
        
        self.t = 0
        
        self.statusFeature = self.statusEmbedding()

    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1


    def statusEmbedding(self):
        statusFeature = torch.zeros(size=(5,env.contentNum)).to(device)
        
        statusFeature[0] = self.v
        statusFeature[1] = self.r[self.u]
        statusFeature[2] = self.p
        statusFeature[3] = self.e
        statusFeature[4] = self.S

        #statusFeature[5] = status['r']
        return statusFeature.T
    
    def getReward(self,lastru,lastp,ru,p,i,action,S,Bu,l_edge,l_cp,e):

        self.Rh =   self.ALPHAh * (torch.log(ru * p + (1-ru) * (1-p)) - torch.log(lastru * lastp + (1-lastru) * (1-lastp)))
        #self.Rh =   self.ALPHAh * torch.log(ru * p + (1-ru) * (1-p))

        self.Ro =   self.BETAo * action[i] * (S[i] / Bu + ( e[i] * l_edge + ( 1-e[i] ) * l_cp ) / S[i])

        self.Rl =   self.BETAl * ( ( 1 - action[i] )  * ( l_cp - ( e[i] * l_edge + ( 1 - e[i] ) * l_cp ) ) ) / S[i]

        self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        return  self.Rh

    def selectAction(self,env,uit,QNetwork,train,memory):

        self.lastStatusFeature = self.statusFeature
        self.lastAction = self.action
        self.lastp = self.p
        self.lastr = self.r

        self.updateViewContent(uit[1])
        self.r , self.p , self.e, self.S, self.l_edge, self.l_cp = env.getStatus()
        self.statusFeature = self.statusEmbedding()
        
        self.reward = self.getReward(self.lastr[self.u],self.lastp,self.r[self.u],self.p,self.W[-1],self.lastAction,self.S,self.Bu,self.l_edge,self.l_cp,self.e)
        
        if train:
            
            lastAction = torch.cat(((1-self.lastAction).unsqueeze(1),self.lastAction.unsqueeze(1)),1)
            memory.push(self.lastStatusFeature, 
                    lastAction.to(device), 
                    self.statusFeature,
                    self.reward.float().to(device),
                    torch.tensor([self.W[-1]]).to(device))
        
        sample = random.random()
        eps_threshold = self.EPS_END + (self.EPS_START - self.EPS_END) *  np.exp(-1. * self.t / self.EPS_DECAY)
        self.t = self.t + 1
        
        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.action[self.W[-1]] = 1
        if  not train or (train and sample > eps_threshold):
            QNetwork.eval()
            with torch.no_grad():
                Q_value = QNetwork(self.statusFeature)
                actionIndex = list((Q_value[:,1]-Q_value[:,0]).argsort()[0:self.Bu])
            QNetwork.train()
        else:
            actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
        if self.W[-1] not in actionIndex:
            actionIndex.pop()
        for index in actionIndex:
            self.action[index] = 1

        env.updateEnv(self.u,self.action.numpy(),uit[2])

        return self.action

In [8]:
BATCH_SIZE = 128
GAMMA = 0.63

def optimize_model():
    if len(memory) < BATCH_SIZE:
        #print(len(memory))
        return 0
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    #non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    #non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    content_batch = torch.cat(batch.content)
    next_state_batch = torch.cat(batch.next_state)
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_mask_bacth = action_batch.ge(0.5)
    state_action_values = torch.masked_select(policy_net(state_batch),state_action_mask_bacth)
    #print(state_action_values,state_action_values.dtype)
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    #next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    def getNextStatusQ(s_batch,c_batch):
        
        Q_value= torch.stack(target_net(s_batch).chunk(BATCH_SIZE,dim=0))
        c = c_batch.chunk(BATCH_SIZE,dim=0)
        action = torch.zeros(size=(BATCH_SIZE,contentNum,2),dtype=int).to(device)
        for b in range(BATCH_SIZE):
            Q_value_sortindex = list((Q_value[b,:,1]-Q_value[b,:,0]).argsort()[0:10])
            i = c[b].squeeze()
            if i not in Q_value_sortindex:
                Q_value_sortindex.pop()
            action[b,i,1] = 1
            for index in Q_value_sortindex:
                action[b,index,1] = 1
        action[:,:,0]=1-action[:,:,1]
        action_mask = action.ge(0.5).to(device)
        next_state_values = torch.masked_select(Q_value,action_mask).float()

        return next_state_values
    
    next_state_values =  getNextStatusQ(next_state_batch,content_batch)

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values.unsqueeze(1), expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

    return loss

In [9]:
# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
#init_screen = get_screen()
#_, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
#n_actions = env.action_space.n
policy_net = DQN(5, 2).to(device)
target_net = DQN(5, 2).to(device)

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters())

memory = ReplayMemory(10000)

In [10]:
MODELPATH =  './model_dict/dnn_l0_'
rewardPara = {"alpha":1,"betao":5,"betal":0}

In [11]:

num_episodes = 10
TARGET_UPDATE = 1
bestReward =  0

env = ENV(userNum,contentNum)
UEs = {}
sumReward = 0
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0


for i_episode in range(num_episodes):
    # Initialize the environment and state

    for index,trace in trainUIT.iterrows():
        uit = trace.to_numpy()
        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)

        ue = UEs[uit[0]]
        
        actionIndex = np.argwhere(ue.lastAction)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,1,memory)

        sumReward += float(ue.reward.sum())
        
        if index % 10 == 0:
            # Perform one step of the optimization (on the policy network)
            loss += float(optimize_model())
        
        if index % 100 == 0:
            # Update the target network, copying all weights and biases in DQN
            target_net.load_state_dict(policy_net.state_dict())

        if index % 10000 == 0:
            psi = 0
            p = torch.from_numpy(env.p)
            for u in UEs:
                psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
            print("Time:",time.asctime( time.localtime(time.time())),"--Episode:",i_episode,"  Index:",index,"  Reward:",sumReward/(index+1),"  Loss:",float(loss/(index+1)))
            print("UEHitrate:",UEHit.sum()/(index+1)," edgeHitrate",edgeHit/(index+1),"sumHitrate",(edgeHit+UEHit.sum())/(index+1)," privacy:",psi/len(UEs))
            
            print()
            # Update the target network, copying all weights and biases in DQN

    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        
    print("----------------------------------------------------------------")
    print("Time:",time.asctime( time.localtime(time.time())),"--End episode:",i_episode,"  Reward:",sumReward/(index+1),"  Loss:",loss/(index+1))
    print("UEHitrate:",UEHit.sum()/(index+1)," edgeHitrate",edgeHit/(index+1),"sumHitrate",(edgeHit+UEHit.sum())/(index+1)," privacy:",psi/len(UEs))
    print("----------------------------------------------------------------")
    print()
    # Update the target network, copying all weights and biases in DQN
    target_net.load_state_dict(policy_net.state_dict())
    

    if sumReward > bestReward:
        bestReward = sumReward
        bestLoss = loss
        bestUEHit = UEHit
        bestEdgeHit = edgeHit
        bestEpisode = i_episode
        bestPath = MODELPATH+'ep{}_'.format(bestEpisode)+time.strftime("%m%d-%H-%M-%S",time.localtime(time.time()))
        torch.save(policy_net.state_dict(),bestPath)
    


    env = ENV(userNum,contentNum)
    UEs = {}
    sumReward = 0
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0

    

Time: Tue Sep 14 00:47:20 2021 --Episode: 0   Index: 0   Reward: 0.0   Loss: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: tensor(2.3921, dtype=torch.float64)

Time: Tue Sep 14 00:50:25 2021 --Episode: 0   Index: 10000   Reward: 1.0417135812212377   Loss: 0.00011844962348165392
UEHitrate: 0.0124987501249875  edgeHitrate 0.29217078292170784 sumHitrate 0.3046695330466953  privacy: tensor(2.1555, dtype=torch.float64)

Time: Tue Sep 14 00:53:13 2021 --Episode: 0   Index: 20000   Reward: 0.48869786186548997   Loss: 6.725125172491725e-05
UEHitrate: 0.015499225038748062  edgeHitrate 0.26538673066346685 sumHitrate 0.28088595570221486  privacy: tensor(2.0178, dtype=torch.float64)

Time: Tue Sep 14 00:56:10 2021 --Episode: 0   Index: 30000   Reward: 0.3515378633093137   Loss: 4.633755886850308e-05
UEHitrate: 0.015366154461517949  edgeHitrate 0.2626912436252125 sumHitrate 0.27805739808673047  privacy: tensor(1.8394, dtype=torch.float64)

Time: Tue Sep 14 00:59:12 2021 --Episode: 0 

In [12]:
policy_net = DQN(5, 2).to(device)
target_net = DQN(5, 2).to(device)
policy_net.load_state_dict(torch.load(bestPath))
policy_net.eval()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()



env = ENV(userNum,contentNum)
UEs = {}
sumReward = 0
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0


for index,trace in trainUIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit,policy_net,0,memory)
    sumReward += ue.reward.sum()
    
    if index % 10000 == 0:
        print("Time:",time.asctime( time.localtime(time.time())),"--Episode:",i_episode,"  Index:",index," Reward:",sumReward/(index+1),)
        print("UEHitrate:",UEHit.sum()/(index+1)," edgeHitrate",edgeHit/(index+1),"sumHitrate",(edgeHit+UEHit.sum())/(index+1))
        print()

Time: Tue Sep 14 03:21:45 2021 --Episode: 9   Index: 0  Reward: tensor(0., dtype=torch.float64)
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0

Time: Tue Sep 14 03:22:09 2021 --Episode: 9   Index: 10000  Reward: tensor(0.2794, dtype=torch.float64)
UEHitrate: 0.024997500249975  edgeHitrate 0.38806119388061194 sumHitrate 0.41305869413058693

Time: Tue Sep 14 03:22:33 2021 --Episode: 9   Index: 20000  Reward: tensor(0.1680, dtype=torch.float64)
UEHitrate: 0.030698465076746163  edgeHitrate 0.39473026348682566 sumHitrate 0.42542872856357183

Time: Tue Sep 14 03:22:57 2021 --Episode: 9   Index: 30000  Reward: tensor(0.1277, dtype=torch.float64)
UEHitrate: 0.031932268924369185  edgeHitrate 0.3646878437385421 sumHitrate 0.39662011266291125

Time: Tue Sep 14 03:23:20 2021 --Episode: 9   Index: 40000  Reward: tensor(0.1194, dtype=torch.float64)
UEHitrate: 0.03202419939501513  edgeHitrate 0.33611659708507285 sumHitrate 0.368140796480088

Time: Tue Sep 14 03:23:43 2021 --Episode: 9   Index: 50000 

In [13]:
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    if u % 100 == 0:
        print(u,psi)

psi/contentNum

100 tensor(176.8964, dtype=torch.float64)
0 tensor(178.8415, dtype=torch.float64)


tensor(0.0382, dtype=torch.float64)